# Esse projeto tem como objetivo analisar e prever os quadros do novo coronavírus aqui no Brasil.

Vamos primeiramente realizar a importação das bibliotecas que serão usadas:

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

Vamos agora carregar os dados que serão utilizados, lembrando de usar o comando `parse_dates` por causa das datas:

In [ ]:
covid = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv', parse_dates = ['ObservationDate', 'Last Update'])
covid

### Realizando um rápida limpeza dos dados, deixando somente letras minúsculas e sem potuações ou espeaços. Tudo isso para uma melhor codificação

In [ ]:
import re
def corrige_colunas(col_name):
    return re.sub(r'[/| ]', '', col_name).lower()

In [ ]:
covid.columns = [corrige_colunas(col) for col in covid.columns]
covid

## Brasil:
Vamos filtrar os dados e obter somente os números do Brasil

In [ ]:
covid_br = covid.loc[covid.countryregion == 'Brazil']
covid_br

FareI uma codificação para melhorar a visualização dos gráficos, irei usar o `groupby`para poder acumular os números de `confirmed`, `deaths`, `recovered`e das novas colunas que serão criadas para estudos posteriores.

In [ ]:
br_sum = covid_br.groupby('observationdate').agg({'confirmed': 'sum', 'deaths': 'sum', 'recovered': 'sum'}).reset_index()
br_sum

## Evolução dos casos de confirmados:
Agora com os dados já filtrados, iremos realizar analises sobre os números da COVID-19 aqui no Brasil, a começar com um gráfico dos casos confirmados.

In [ ]:
px.line(br_sum, "observationdate", "confirmed", title = 'Quadro Evolutivo do Número de Confirmados')

O gráfico infelizmente mostra que desde o início da pandemia houve crescimento e mesmo passados tantos meses, os números só aumentam. Chegamos a uma marca de mais de **quatro milhões** de infectados. 

### Calculando o número de novos casos por dia:
Vou criar uma função que irá retornar o valor de novos casos por dia, por exemplo, 10/09/2020 tivemos mais 40557 novos casos, ou seja, 40557 novas pessoas foram infectadas com o coronavírus. Vamos realizar esse cálculo para número de mortes e recuperados também, porém vamos começar pelo número de confirmados:

In [ ]:
br_sum['novoscasos'] = list(map(
lambda x: 0 if (x == 0) else br_sum['confirmed'].iloc[x] - br_sum['confirmed'].iloc[x-1],
    np.arange(br_sum.shape[0])
))

In [ ]:
br_sum

Visualizando os números de forma gráfica:

In [ ]:
px.line(br_sum, x = "observationdate", y ="novoscasos", title = 'Quadro Evolutivo de Novos Casos Diários')

É preocupante ver que diariamente os números variam bastante. Enquanto no dia x tivemos uma queda de pessoas com COVID-19, o dia y mostra uma alta significativa, as vezes três vezes mais. É uma instabilidade muito grande. Esse gráfico explicita que o Brasil não tem controle sobre os casos do vírus e que estamos longe de uma recuperação. Infelizmente a flexibilização da quarentena em alguns Estados já mostram que estamos longe de conseguir controlar esses números. As variações são bem diferentes uma da outra, é como se o país sempre estivesse numa eterna montanha russa: uma hora sobe, em outra desce.

## Evolução do número de mortes:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = br_sum.observationdate, y = br_sum.deaths, 
                         name = 'Número de Mortos', mode = 'lines', 
                         line = {'color':'red'}))
fig.update_layout(title = 'Quadro Evolutivo do Número de Mortes')
fig.show()

Enquanto nós temos mais de **quatro milhões** de brasileiros infectados pelo vírus, o Brasil apresenta mais de **cento e trinta mil** mortos pela COVID-19. Felizmente é uma proporção menor se comparado ao quadro de confirmados, mas é um número bem elevado. Mesmo com o passar dos meses, o gráfico aponta que só houve alta no número de mortos, nunca ocorreu uma redução da curva, isso mostra uma falha no sistema brasileiro ao não conseguir controlar o contágio e consequentimente as mortes decorrentes dos coronavírus. Sendo um descaso ao familiares que perderam seus entes queridos, [declarações do Presidente Jair Bolsonaro mostra o quão ele considera irrelevante as pessoas estarem morrendo de COVID-19](https://noticias.uol.com.br/saude/ultimas-noticias/redacao/2020/06/02/bolsonaro-a-gente-lamenta-todos-os-mortos-mas-e-o-destino-de-todo-mundo.htm). Até mesmo os brasileiros que não entendem a seriedade da doença e acabam não se cuidado da maneira que é recomendada: furando quarentena mesmo que não sendo necessário, não usando máscara (ou usando de maneira incorreta), entre outras precauções não tomadas pelo cidadão e aumentando o risco de mais pessoas contaminadas e mortas.

### Calculando o número de mortes por dia:

In [ ]:
br_sum['novasmortes'] = list(map(
lambda x: 0 if (x == 0) else br_sum['deaths'].iloc[x] - br_sum['deaths'].iloc[x-1],
    np.arange(br_sum.shape[0])
))

br_sum

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = br_sum.observationdate, y = br_sum.novasmortes, 
                         name = 'Mortes Diárias', mode = 'lines+markers', 
                         line = {'color':'blue'}))
fig.update_layout(title = 'Quadro Evolutivo do Número de Mortes Diárias')
fig.show()

A mesma instabilidade se repete aqui, é preocupante saber que em um dia podemos ter baixos casos de pessoas mortas pelo COVID-19 e no outro um quadro muito mais de pessoas que morreram pela doença. Felizmente os primeiros dias de setembro mostram um quadro onde reduzimos para menos de 1000 mortos por dia pelo coronavírus, o que é uma vitória se compararmos ao números maiores que 1400 mortos por dias nos meses anteriores. Apesar de valores mais baixos, ainda são números altos e merecem mais atenção.

## Evolução do número de recuperados:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = br_sum.observationdate, y = br_sum.recovered, 
                         name = 'Número de Recuperados', mode = 'lines', 
                         line = {'color':'orange'}))
fig.update_layout(title = 'Quadro Evolutivo do Número de Recuperados')
fig.show()

Um gráfico que traz felicidades: há mais recuperados do que mortos. Temos mais de **três milhões** de pessoas que se recuperam do vírus. Felizmente com a produção da vacina e com mais pesquisas sobre a doença, é capaz desses números aumentarem e superarem também o número de infectados.

### Calculando o número de recuperados por dia:

In [ ]:
br_sum['novosrecuperados'] = list(map(
lambda x: 0 if (x == 0) else br_sum['recovered'].iloc[x] - br_sum['recovered'].iloc[x-1],
    np.arange(br_sum.shape[0])
))

br_sum

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = br_sum.observationdate, y = br_sum.novosrecuperados, 
                         name = 'Número de Recuperados', mode = 'lines', 
                         line = {'color':'green'}))
fig.update_layout(title = 'Quadro Evolutivo do Número de Recuperados Por Dia')
fig.show()

Sem surpresas, o quadro de instabilidade se repete pela terceira vez. Aqui as variações conseguem ser mais gritantes que as anteriores. Variações mais gritantes porque não há padrões nessa figura. Se voltarmos para os gráficos diários de mortos e confirmados, percebemos que eles seguem um padrão de alta e baixa dos valores. Algo que não acontece nesse gráfico. Diferentes dos outros, aqui é triste ver que há dias em que temos uma queda do número de pessoas que se recuperaram da doença.

# Taxa de Crescimento da COVID-19 no Brasil
Vamos criar uma função para calcular a taxa de crscimento

In [ ]:
def taxa_crescimento(data, variable, data_inicio = None, data_fim = None):
    if data_inicio == None:
        data_inicio = data.observationdate.loc[data[variable] > 0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)
        
    if data_fim == None:
        data_fim = data.observationdate.iloc[-1]
    else:
        data_fim = pd.to_datetime(data_fim)
    
    passado = data.loc[data.observationdate == data_inicio, variable].values[0]
    presente = data.loc[data.observationdate == data_fim, variable].values[0]
    
    n = (data_fim - data_inicio).days
    
    taxa = (presente/passado)**(1/n) - 1
    
    return taxa*100

### Taxa de Crescimento médio desde o início da pandemia no Brasil

In [ ]:
taxa_crescimento(br_sum, 'confirmed')

Ou seja, o COVID-19 cresceu no Brasil 7,94% ao dia desde o inicio da pandemia.

### Taxa de Crescimento médio diário (quanto cresceu de um dia para o outro)

In [ ]:
def taxa_crescimento_diario(data, variable, data_inicio = None):
    if data_inicio == None:
        data_inicio = data.observationdate.loc[data[variable] > 0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)
        
    data_fim = data.observationdate.max()
    
    n = (data_fim - data_inicio).days
    
    taxa = list(map(lambda x: (data[variable].iloc[x] - data[variable].iloc[x-1]) / data[variable].iloc[x-1],
                   range(1, n+1)))
    
    return np.array(taxa)*100

In [ ]:
tx = taxa_crescimento_diario(br_sum, 'confirmed')
tx

Vamos visualizar essas taxas em um gráfico:

In [ ]:
primeiro_dia = br_sum.observationdate.loc[br_sum.confirmed > 0 ].min()
px.line(x = pd.date_range(primeiro_dia, br_sum.observationdate.max())[1:], y = tx, title = 'Taxa de Crescimento diário da COVID-19 no Brasil')

O início do gráfico é bem preocupante, mostrando que havia alto índice de contágio no inicio da pandemia. A partir do mês de Abril se houve uma queda significativa da taxa de contágio. Mas também temos que considerar que os números estão tão elevador que atualmente já não são tão distantes como no inicio da pandemia. Por exemplo, lá em Março tínhamos no dia x 300 pessoas contaminadas, no dia y tínhamos mais 700 novas pessoas contaminadas, ou seja, cresceu mais da metade o número de pessoas infectadas. Chegando em Agosto, no dia w temos 40000 novos casos e no dia seguinte temos 40020 novos infectados, ou seja, a diferença dos novos casos é menor de um dia para outro, fazendo com que a taxa diminua ao longo do gráfico.

# Prevendo os próximos números da COVID-19 no Brasil

## Casos Confirmados:

In [ ]:
confirmados = br_sum.confirmed
confirmados.index = br_sum.observationdate
confirmados

In [ ]:
!pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima
modelo = auto_arima(confirmados)

In [ ]:
fig = go.Figure(go.Scatter(x = confirmados.index, y = confirmados, name = 'Observados'))
fig.add_trace(go.Scatter(x = confirmados.index, y = modelo.predict_in_sample(), name = 'Previsores'))
fig.add_trace(go.Scatter(x = pd.date_range('2020-09-16', '2020-10-01'), y = modelo.predict(18), name = 'Previsões'))
fig.update_layout(title = 'Previsão para o número de confirmados para os próximos 15 dias')
fig.show()

Segundo o gráfico, a previsão é que cheguemos no mês de outubro com mais de **quatro milhões e setecentos mil** infecatdos pelo novo coronavírus.

### Decompondo a série temporal:

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

In [ ]:
#confirmados = confirmados.sort_index(inplace= True)

In [ ]:
decom = seasonal_decompose(x = confirmados, period = 101)

In [ ]:
fig, (ax1,ax2,ax3, ax4) = plt.subplots(4,1, figsize=(15,8))
decom.observed.plot(ax=ax1)
decom.trend.plot(ax=ax2)
decom.seasonal.plot(ax=ax3)
decom.resid.plot(ax=ax4)
plt.tight_layout()

O gráfico acima msotra a decomposição da Série Temporal do número de confirmados infectados pelo novo cornavírus. Podemos ver na terceira figura certos pontos que são anormais, período de de Maio e mês de Agosto. Esses meses apresentaram sazonalidades, no período de Maio para Junho é provável que seja por causa da [medida tomada de acabar com as divulgações de dados sobre a COVID-19](https://www.cnnbrasil.com.br/politica/2020/06/05/bolsonaro-divulgacao-de-numeros-da-covid-19-as-22h-busca-dados-consolidados) aqui no Brasil. Agosto eu aposto para o início da [flexibilização da quarentena](https://www.bbc.com/portuguese/brasil-53619167) em certos Estados do país.








# Calculando o ponto de virada: quando a curva poderá começar a diminuir

In [ ]:
#!conda install -c conda-forge fbprophet -y

In [ ]:
from fbprophet import Prophet

Com essa biblioteca é necessário realizar alguns preprocessamentos, então vamos realiza-los. Vamos criar a variavel `train`e `test`para treinar e testar o modelo. Para treino serão usados 80% dos dados e os demais serão para teste.

In [ ]:
train = confirmados.reset_index()[:-5]
test = confirmados.reset_index()[-5: ]

Por padrões da biblioteca será necessário criar um novo nome para as colunas.

In [ ]:
train.rename(columns = {'observationdate': 'ds', 'confirmed': 'y'}, inplace = True)
test.rename(columns = {'observationdate': 'ds', 'confirmed': 'y'}, inplace = True)

Criando agora o modelo:
- Considerando que 1/3 população brasileira será contaminada, segundo o IBGE hoje dia 15/09, o Brasil tem [212050850](https://www.ibge.gov.br/apps/populacao/projecao/box_popclock.php) de população.

In [ ]:
pred = Prophet(growth = 'logistic', changepoints = ['2020-04-15', '2020-06-01', '2020-07-25'])

In [ ]:
pop = 70683616
train['cap'] = pop

In [ ]:
pred.fit(train)

In [ ]:
datas_futuras = pred.make_future_dataframe(periods = 830)
datas_futuras['cap'] = pop
forecast = pred.predict(datas_futuras)

Vendo os resultados em gráfico:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = forecast.ds, y = forecast.yhat, name = 'Previsões Futuras'))
fig.add_trace(go.Scatter(x = train.ds, y = train.y, name = 'Treino do Modelo'))
fig.update_layout(title = 'Previsão para o ponto de viarada dos quadros da COVID-19 no Brasil')
fig.show()

Chocante! Considerando que um terço da população brasileira seja contaminada, a curva só começará a achatar lá para 2022. Claro que assim não estão sendo consideradas a evolução da vacina e outros possíveis medicamentos comprovados de eficácia. É preocupante imaginar que se não houver vacina ou medicamentos, o Brasil só poderá se recuperar completamente da pandemia só depois de 2022. Segundo o gráfico e não considerando vacina ou avanços dos estudos, o número de infectados continuará a crescer até 2021 e irá se tornar estável somente em 2022.

## Número de Mortes:

In [ ]:
mortes = br_sum.deaths
mortes.index = br_sum.observationdate
mortes

In [ ]:
modelo1 = auto_arima(mortes)

In [ ]:
fig = go.Figure(go.Scatter(x = mortes.index, y = mortes, name = 'Observados'))
fig.add_trace(go.Scatter(x = mortes.index, y = modelo1.predict_in_sample(), name = 'Previsores'))
fig.add_trace(go.Scatter(x = pd.date_range('2020-09-16', '2020-10-01'), y = modelo1.predict(18), name = 'Previsões'))
fig.update_layout(title = 'Previsão para o número de Mortos para os próximos 15 dias')
fig.show()

Segundo o gráfico, a previsão é que cheguemos no mês de outubro com mais de cento e quarenta mil mortos pelo novo coronavírus.

## Número de Recuperados:

In [ ]:
recuperados = br_sum.recovered
recuperados.index = br_sum.observationdate
recuperados

In [ ]:
modelo2 = auto_arima(recuperados)

In [ ]:
fig = go.Figure(go.Scatter(x = recuperados.index, y = recuperados, name = 'Observados'))
fig.add_trace(go.Scatter(x = recuperados.index, y = modelo2.predict_in_sample(), name = 'Previsores'))
fig.add_trace(go.Scatter(x = pd.date_range('2020-09-16', '2020-10-01'), y = modelo2.predict(18), name = 'Previsões'))
fig.update_layout(title = 'Previsão para o número de Recuperados para os próximos 15 dias')
fig.show()

Segundo o gráfico, a previsão é que cheguemos no mês de outubro com mais de **quatro milhões e duzentos mil** recuperados do novo coronavírus.

# Previsão geral para os próximos 15 dias da COVID-19 aqui no Brasil

In [ ]:
fig = go.Figure(go.Scatter(x = pd.date_range('2020-09-16', '2020-10-01'), y = modelo.predict(18), name = 'Previsões Confirmados'))
fig.add_trace(go.Scatter(x = pd.date_range('2020-09-16', '2020-10-01'), y = modelo1.predict(18), name = 'Previsões Mortos'))
fig.add_trace(go.Scatter(x = pd.date_range('2020-09-16', '2020-10-01'), y = modelo2.predict(18), name = 'Previsões Recuperados'))
fig.update_layout(title = 'Previsão geral dos números da COVID-19 para os próximos 15 dias')
fig.show()

Por fim esse gráfico que nos traz a previsão dos números gerais da COVID-19 aqui no Brasil. Felizmente o Número de recuperados chega perto do número de infectados. O número de mortos continua em patamares menores. Podemos concluir que a pandemia está longe de acabar, mas o preocupante é saber que os prórprios brasileiros não se importam em tomar os cuidados necessários para reduzir os números do vírus. O Governo Federal tão pouco tomou medidas que pudessem deixar mais controlados esses números, muitas medidas que foram tomadas em outros países foram ignoradas aqui no nosso. O próprio presidente fez pouco caso da doença, [tratando-a como "gripezinha" e que é o "destino de todos morrer"](https://oglobo.globo.com/brasil/da-gripezinha-ao-dai-confira-as-reacoes-de-bolsonaro-enquanto-aumentavam-as-mortes-pela-pandemia-no-brasil-24402593). Sempre quando perguntado sobre o quadro da doença no Brasil, o Bolsonaro se mostra desacreditado da seriedade da situação e [não toma cuidados](https://www.bbc.com/portuguese/brasil-53331958) recomendados pela Organização Mundial da Saúde (OMS). 

Infelizmente não é só o presidente do país que segue o mal exemplo, muitos dos brasileiros também não tomam as medidas de segurança contra o vírus. [Muitos furaram a quarentena](https://www1.folha.uol.com.br/cotidiano/2020/04/28-dos-brasileiros-nao-fazem-isolamento-contra-coronavirus-diz-datafolha.shtml) sem necessidade, depois da reabertura dos estabelecimentos, as aglomerações voltaram a ser vistas. Houve também muitas [festas clandestinas](https://www.cnnbrasil.com.br/nacional/2020/08/01/festas-clandestinas-acontecem-em-meio-a-pandemia-pelo-pais), incluindo de pessoas famosas e influentes como o caso da [Gabriela Pugliese](https://g1.globo.com/pop-arte/noticia/2020/04/27/gabriela-pugliesi-tem-contratos-suspensos-apos-fazer-festa-durante-quarentena.ghtml), entre outras organizações de eventos que [não respeitam o distanciamento e demais medidas de segurança](https://veja.abril.com.br/saude/covid-19-festas-reunem-milhares-sem-cuidado-com-o-contagio/). O não respeito a essas medidas não auxiliam para uma previsão positiva dos números da COVID-19 aqui no Brasil.

As previsões para os próximos dias é de **alta**, tanto para o número de novos casos, mortes e felizmente recuperados. Como falado anteriormente, o Brasil não é um bom exemplo quanto as regras de segurança para a prevenção e diminuição do contágio do novo coronavírus. Segundo o meus estudos, tudo indica que o Brasil está longe de uma recuperação da pandemia mas as pessoas não se importam mais e continuam buscando seguir suas vidas como se não estivesse ocorrendo uma pandemia em pleno globo. 
Segundo os gráficos estudados, até 01/10/2020 nós teremos:
- 4.762.618 INFECTADOS;
- 143.816,5 MORTOS;
- 4.234.156 RECUPERADOS;